In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import optimize
%matplotlib inline
from sklearn.model_selection import train_test_split

## Data Preprocessing

In [2]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
'''
train = pd.read_csv('data/digit-recognizer/train.csv')
test = pd.read_csv('data/digit-recognizer/test.csv')

#Complete Training data 
y_tr = train['label'].values
X_tr = train.drop(columns=['label']).values/255

#Test data fot prediction
X_test = test.values/255

#Splitting data 
X_train = X_tr[:30000]
y_train = y_tr[:30000]
test_x = X_tr[30000:42000]
test_y = y_tr[30000:42000]

def process_data():
    #Trainig Data
    training_inputs = [np.reshape(x, (784, 1)) for x in X_train]
    training_results = y_train#[vectorized_result(y) for y in train_y]
    training_data = list(zip(training_inputs, training_results))
    #Test Data
    test_inputs = [np.reshape(x, (784, 1)) for x in test_x]
    test_data = list(zip(test_inputs, test_y))
    return (training_data, test_data)

training_data , test_data = process_data() 
'''
print("Data for reference")

Data for reference


In [4]:
import pickle
import gzip
def load_data():
    f = gzip.open('/home/pratik/ml_workspace/Machine_Learning/data/mnist.pkl.gz', 'rb')
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()
    f.close()
    return p
 
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

training_data, validation_data, test_data = load_data_wrapper()

In [5]:
"""fig, axes = plt.subplots(2,5, figsize=(10,5))
axes = axes.flatten()
idx = np.random.randint(0,30000,size=10)
for i in range(10):
    axes[i].imshow(X_train[idx[i],:].reshape(28,28), cmap='gray')
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title(str(int(y_train[idx[i]])), color= 'black', fontsize=25)
plt.show()"""
print(" ")

In [6]:
"""print("----Traininig data----")
y_value=np.zeros((1,10))
for i in range (10):
    print("Occurance of ",i,"=",np.count_nonzero(y_train==i))
    y_value[0,i-1]= np.count_nonzero(y_train==i)"""
print(" ")

In [7]:
"""y_value=y_value.ravel()
x_value=[0,1,2,3,4,5,6,7,8,9]

plt.xlabel('label')
plt.ylabel('count')
plt.bar(x_value,y_value,0.7)"""
print(" ")

# Neural Network model

In [8]:
class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        return (a-y) * sigmoid_prime(z)

In [9]:
class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        return (a-y)

In [10]:
class NeuralNetwork(object):
    
    def __init__(self, sizes,cost=CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) 
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        self.cost = cost
        
    def sigmoid(self,x):
        s  = 1/(1+np.exp(-x))
        return s

    def sigmoidPrime(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def forward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a
    
    def backprop(self, x, y):
        b_up = [np.zeros(b.shape) for b in self.biases]
        w_up = [np.zeros(w.shape) for w in self.weights]
        
        #Feed forward
        activation = x
        activations = [x] 
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        
        #Backpass
        delta = self.cost_derivative(activations[-1], y)*self.sigmoidPrime(zs[-1])
        b_up[-1] = delta
        w_up[-1] = np.dot(delta, activations[-2].transpose())
        
        #Updation terms
        for l in range(2, self.num_layers):
            z = zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta)*self.sigmoidPrime(z)
            b_up[-l] = delta
            w_up[-l] = np.dot(delta, activations[-l-1].transpose())
        
        return (b_up, w_up)
        
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print ("---------Epoch %s training complete---------" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print ("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print ("Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print ("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print ("Accuracy on evaluation data: {} / {}"\
                       .format(self.accuracy(evaluation_data), n_data))
                
        #return (evaluation_cost, evaluation_accuracy,\
        #                training_cost, training_accuracy)
                
    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        
        b_up = [np.zeros(b.shape) for b in self.biases]
        w_up = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_b_up, delta_w_up = self.backprop(x, y)
            b_up = [bu+dnb for bu, dnb in zip(b_up, delta_b_up)]
            w_up = [wu+dnw for wu, dnw in zip(w_up, delta_w_up)]
        
        #Updating the weights and biases
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*wu 
                        for w, wu in zip(self.weights, w_up)]
        self.biases = [b-(eta/len(mini_batch))*bu 
                       for b, bu in zip(self.biases, b_up)] 
    
    def accuracy(self, data, convert=False):
        if convert:
            results = [(np.argmax(self.forward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.forward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)
    
    def total_cost(self, data, lmbda, convert=False):
        cost = 0.0
        for x, y in data:
            a = self.forward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost 

In [11]:
nn = NeuralNetwork([784,100,30,10])

In [13]:
nn.SGD(training_data, 10, 10, 3.0, 1, evaluation_data=test_data,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=True,
            monitor_training_cost=False,
            monitor_training_accuracy=False)

---------Epoch 0 training complete---------
Accuracy on evaluation data: 9582 / 10000
---------Epoch 1 training complete---------
Accuracy on evaluation data: 9715 / 10000
---------Epoch 2 training complete---------
Accuracy on evaluation data: 9705 / 10000
---------Epoch 3 training complete---------
Accuracy on evaluation data: 9750 / 10000
---------Epoch 4 training complete---------
Accuracy on evaluation data: 9739 / 10000
---------Epoch 5 training complete---------
Accuracy on evaluation data: 9649 / 10000
---------Epoch 6 training complete---------
Accuracy on evaluation data: 9742 / 10000
---------Epoch 7 training complete---------
Accuracy on evaluation data: 9716 / 10000
---------Epoch 8 training complete---------
Accuracy on evaluation data: 9731 / 10000
---------Epoch 9 training complete---------
Accuracy on evaluation data: 9749 / 10000
